<a href="https://colab.research.google.com/github/asgardian1196/asg-ml/blob/main/CNN_Olivetti_Faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Fetching Olivetti Dataset

In [1]:
from sklearn import datasets
faces = datasets.fetch_olivetti_faces()
faces.data.shape

downloading Olivetti faces from https://ndownloader.figshare.com/files/5976027 to /root/scikit_learn_data


(400, 4096)

### Creating test-train split

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(faces.data, faces.target, random_state=0)

print(X_train.shape, X_test.shape)

(300, 4096) (100, 4096)


### Reshaping data for CNN 

In [3]:
X_train = X_train.reshape(-1,64,64,1)
X_test = X_test.reshape(-1,64,64,1)

print("X_train: ",X_train.shape)
print("X_test: ",X_test.shape)


X_train:  (300, 64, 64, 1)
X_test:  (100, 64, 64, 1)


### One-Hot encoding for target data

In [4]:
from keras.utils.np_utils import to_categorical 

y_train_ = to_categorical(y_train, num_classes = 40)
y_test_ = to_categorical(y_test, num_classes = 40)

print("y_train_ shape: ",y_train_.shape)
print("y_test_ shape: ",y_test_.shape)

y_train_ shape:  (300, 40)
y_test_ shape:  (100, 40)


### Imports

In [5]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau


### Creating the sequential model

In [6]:
model = Sequential()


### Adding Layers

In [7]:
# Adding a 2D convolution layer with 20 filters

model.add(Conv2D(filters = 20, 
                 kernel_size = (5,5),
                 padding = 'Same', 
                 activation ='relu',
                 input_shape = (64,64,1)))

In [8]:
# Max pooling operation for 2D spatial data.
model.add(MaxPool2D(pool_size=(2,2)))

In [9]:
# Apply Dropout to the input.
model.add(Dropout(0.25))

In [10]:
# Adding another 2D convolution layer, with 50 filters
model.add(Conv2D(filters = 50,
                 kernel_size = (6,6),
                 padding = 'Same', 
                 activation ='relu'))

In [11]:
# MaxPool and dropout
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

In [12]:
# Adding next 2D convolution layer with 150 filters
model.add(Conv2D(filters = 150,
                 kernel_size = (5,5),
                 padding = 'Same', 
                 activation ='relu', 
                 input_shape = (64,64,1)))

In [13]:
# Flatten the layer
# Keras.layers.flatten function flattens the multi-dimensional input tensors into a single dimension

model.add(Flatten())


In [14]:
# Dense Layer 
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(40, activation = "softmax"))

### Optimizer

In [15]:
optimizer = RMSprop(learning_rate=0.001, 
                    rho=0.9, 
                    epsilon=1e-08,
                    decay=0.0)

In [16]:
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])


Image Data Generator

In [17]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=5,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.05, # Randomly zoom image 
        width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

In [18]:
datagen.fit(X_train)

In [19]:
batch_size = 20
epoch = 37

history = model.fit(datagen.flow(X_train,y_train_, batch_size=batch_size),
                    epochs = epoch, 
                    validation_data = (X_test,y_test_),
                    verbose = 2, 
                    steps_per_epoch=X_train.shape[0] // batch_size                             )

Epoch 1/37
15/15 - 11s - loss: 4.4905 - accuracy: 0.0067 - val_loss: 3.6917 - val_accuracy: 0.0100 - 11s/epoch - 744ms/step
Epoch 2/37
15/15 - 0s - loss: 3.6913 - accuracy: 0.0267 - val_loss: 3.6937 - val_accuracy: 0.0100 - 250ms/epoch - 17ms/step
Epoch 3/37
15/15 - 0s - loss: 3.6904 - accuracy: 0.0300 - val_loss: 3.7016 - val_accuracy: 0.0100 - 238ms/epoch - 16ms/step
Epoch 4/37
15/15 - 0s - loss: 3.7189 - accuracy: 0.0433 - val_loss: 3.6968 - val_accuracy: 0.0000e+00 - 230ms/epoch - 15ms/step
Epoch 5/37
15/15 - 0s - loss: 3.7006 - accuracy: 0.0200 - val_loss: 3.6977 - val_accuracy: 0.0100 - 244ms/epoch - 16ms/step
Epoch 6/37
15/15 - 0s - loss: 3.6733 - accuracy: 0.0467 - val_loss: 3.6769 - val_accuracy: 0.0200 - 230ms/epoch - 15ms/step
Epoch 7/37
15/15 - 0s - loss: 3.6207 - accuracy: 0.0867 - val_loss: 3.5979 - val_accuracy: 0.0700 - 244ms/epoch - 16ms/step
Epoch 8/37
15/15 - 0s - loss: 3.4231 - accuracy: 0.0967 - val_loss: 3.3013 - val_accuracy: 0.1600 - 245ms/epoch - 16ms/step
Epoc

### Predicting for test data

In [20]:
import numpy as np
y_pred = np.argmax(model.predict(X_test), axis=-1)

y_test = y_test.reshape(-1,)

diff = y_test - y_pred
diff = diff.reshape(-1,1)

true = 0
for i in range(0,len(diff)):
    if diff[i] == 0:
        true = true + 1

Cnn_accuracy = round(100*true/len(diff),2)

print("Cnn_accuracy is %", Cnn_accuracy)

4/4 [==============================] - 0s 4ms/step
Cnn_accuracy is % 97.0


### Comparing with PCA approach

The PCA approach had 83% accuracy whereas CNN has 97% accuracy. CNN was faster and more efficient as compared to the dimensionality reduction based approach